<a href="https://colab.research.google.com/github/fovi-llc/refactor-python/blob/main/konwinski-dspy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab notebook is a copy of the [NeurIPS HackerCup AI Competition (HAC) 2024](https://hackercupai.github.io/) [DSPy](https://github.com/stanfordnlp/dspy) coding example by [Krista Opsahl-Ong](https://www.linkedin.com/in/krista-opsahl-ong-86b096103/), PhD candidate at Stanford hacking on DSPy, from here: https://github.com/stanfordnlp/dspy/tree/main/examples/coding

Weights & Biases interview of Krista about this example:
https://youtu.be/gpe-rtJN8z8

NeurIPS HackerCup AI Competition (HAC) 2024:
https://hackercupai.github.io/

https://www.facebook.com/codingcompetitions/hacker-cup/2024

Hacker Cup AI dataset at Hugging Face:
https://huggingface.co/datasets/hackercupai/hackercup

DSPy:
https://github.com/stanfordnlp/dspy


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

os.environ["DSP_NOTEBOOK_CACHEDIR"] = "./cache"

import pkg_resources # Install the package if it's not installed
if "dspy-ai" not in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai==2.4.17
    !pip install "openai>1,<2"

import dspy

./cache/compiler


In [ ]:
from google.colab import userdata
# os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")

len(openai.api_key)

51

In [ ]:
import datasets
import os
import openai
import dspy
from dspy import InputField, OutputField, Signature
from dspy import Example
from dspy.evaluate.evaluate import Evaluate
from dspy.teleprompt.random_search import BootstrapFewShotWithRandomSearch
from datetime import datetime
import random
from dspy.teleprompt import MIPROv2
# from hackercup_utils import extract_code, run, check_solution

In [ ]:
import re
import asyncio
import multiprocessing
import concurrent.futures
from typing import List, Optional

"""
Note that this code is largely based off of the code here:
https://github.com/HackerCupAI/starter-kits/blob/main/submit_first_solution/01_one_shot.py

by @tcapelle, with some adaptations for this workflow.
"""

def extract_code(code_str):
    # Regex pattern to extract the code between ```python and ```
    pattern = r"```python\s*([\s\S]*?)\s*```"

    # Use re.search to find the code inside the code block
    match = re.search(pattern, code_str)

    if match:
        # Extract the matched group (the code part inside ```python ... ```)
        code = match.group(1).strip()
    else:
        # Fallback: Assume we still need to strip ``` and ```python if match fails
        code = code_str

    # Remove any leading/trailing ``` and ```python manually, in case the input doesn't match exactly
    code = re.sub(r"^```python\s*", "", code)  # Remove starting ```python
    code = re.sub(r"^```", "", code)  # Remove starting ```
    code = re.sub(r"```$", "", code)  # Remove ending ``

    return code.strip()


import traceback


def run_with_timeout(code: str, input: Optional[str], timeout: int):
    def target_fn(input, return_dict):
        vars = {}
        try:
            exec(code, vars)
            fn = vars.get("solve", lambda x: x)
            return_dict["result"] = fn(input)
        except Exception as e:
            return_dict["error"] = str(e)
            return_dict["stack_trace"] = (
                traceback.format_exc()
            )  # Capture the stack trace

    manager = multiprocessing.Manager()
    return_dict = manager.dict()

    process = multiprocessing.Process(target=target_fn, args=(input, return_dict))
    process.start()
    process.join(timeout)  # Wait for the process to finish or timeout

    if process.is_alive():
        process.terminate()
        process.join()
        return {
            "error": f"Execution exceeded the timeout of {timeout} seconds",
            "result": None,
            "stack_trace": None,
        }

    if "error" in return_dict:
        # Return the error and the stack trace for feedback
        return {
            "error": return_dict["error"],
            "result": None,
            "stack_trace": return_dict.get("stack_trace"),
        }

    return {"result": return_dict.get("result"), "error": None, "stack_trace": None}


async def arun(
    code: Optional[str] = None, input: Optional[str] = None, timeout: int = 60
):
    loop = asyncio.get_running_loop()
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = loop.run_in_executor(
                executor, run_with_timeout, code, input, timeout
            )
            result_dict = await asyncio.wait_for(future, timeout=timeout)
        return result_dict
    except asyncio.TimeoutError:
        return {
            "error": f"Function call timed out after {timeout} seconds",
            "result": None,
            "stack_trace": f"Function call timed out. Code needs to be more efficient.",
        }
    except Exception as e:
        return {"error": str(e), "result": None}


# Function to run code synchronously
def run(code: Optional[str] = None, input: Optional[str] = None, timeout: int = 5):
    return asyncio.run(arun(code, input, timeout))

# Function to check the solution
def check_solution(expected: str, actual: str) -> dict:
    "Check the solution against the expected output"
    matches = 0
    expected_lines = expected.strip().split("\n")
    actual_lines = actual.strip().split("\n")
    offending_cases = []
    for expected_line, actual_line in zip(expected_lines, actual_lines):
        expected_line = expected_line.strip()
        actual_line = actual_line.strip()

        if expected_line == actual_line:
            matches += 1  # +1 for the whole line match
        else:
            offending_cases.append((expected_line, actual_line))
    return {
        "matches": matches == len(expected_lines),
        "total": len(expected_lines),
        "offending_cases": offending_cases,
    }

In [ ]:
### DEFINING HELPER FUNCTIONS ###

def format_mistakes(solution_results):
    offending_cases = ""
    for offending_case in solution_results["offending_cases"]:
        offending_cases += f"Incorrect Output: {offending_case[1]} -> Expected Output: {offending_case[0]}\n"
    return offending_cases


def get_expected_behavior_str(sample_input, sample_output):
    return f"""
input = {str(sample_input)} # input is of type {type(sample_input)}
output = solve(input)
print(output) # Output is of type string
# Prints: {sample_output}
"""


### DEFINE SIMPLE PIPELINE ###

class GenerateCodeSignature(Signature):
    """You are an expert problem solver. Your task is creating the code to solve the problem at hand in python.

    The program should have a single `solve` method that has the following signature:
    input: [str]: The same Input provided above
    output [str]: The same Output provided above

    Here's an example of the format we'd expect for a simple python program that adds 1 to a number:
    ```python def solve(x: int):\n    return x + 1```

    Note:
    * Do NOT print the Output, instead return it.
    * Make sure that your proposed solution is both time and memory efficient.
    """

    problem_description: str = InputField(format=str)
    expected_behavior: str = InputField(format=str)
    solution: str = OutputField(
        format=str, desc="A plan for how we should go about solving this problem."
    )
    python_program: str = OutputField(format=str)


class SimpleGenerateCode(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_code = dspy.Predict(GenerateCodeSignature)

    def forward(self, problem_description, sample_input, sample_output):
        expected_behavior = get_expected_behavior_str(sample_input, sample_output)
        python_code = extract_code(
            self.generate_code(
                problem_description=problem_description,
                expected_behavior=expected_behavior,
            ).python_program
        )

        return dspy.Prediction(solution=python_code)


### DEFINE ADVANCED PIPELINE ###

class GenerateCode(dspy.Module):
    def __init__(self, max_tries=2, num_ensembles=3):
        super().__init__()
        # Initialize variables
        self.max_tries = max_tries
        self.num_ensembles = num_ensembles

        # Initialize layers
        self.generate_code = dspy.ChainOfThought("problem_description, expected_behavior -> python_program", n=self.num_ensembles)
        self.fix_code = dspy.ChainOfThought("problem_description, current_code, expected_behavior, current_incorrect_results -> fixed_code")

    def forward(self, problem_description, sample_input, sample_output):

        expected_behavior = get_expected_behavior_str(sample_input, sample_output)
        solutions = self.generate_code(
            problem_description=problem_description, expected_behavior=expected_behavior
        )
        python_solutions = [
             extract_code(solution.python_program) for solution in solutions.completions
        ]

        for i, python_code in enumerate(python_solutions):
            for try_iter in range(self.max_tries):
                # Test our generated code, get a result
                result_dict = run(code=python_code, input=sample_input, timeout=5)
                error, result, stack_trace = (
                    result_dict["error"],
                    result_dict["result"],
                    result_dict["stack_trace"],
                )
                if error:  # Running code led to an exception, fix code
                    python_code = extract_code(
                        self.fix_code(
                            problem_description=problem_description,
                            current_code=python_code,
                            expected_behavior=expected_behavior,
                            current_incorrect_results=stack_trace,
                        ).fixed_code
                    )
                elif result is None:  # Nothing was returned by program
                    python_code = extract_code(
                        self.fix_code(
                            problem_description=problem_description,
                            current_code=python_code,
                            expected_behavior=expected_behavior,
                            current_incorrect_results="Nothing was returned!",
                        ).fixed_code
                    )
                elif not isinstance(result, str):  # Wrong type returned by program
                    python_code = extract_code(
                        self.fix_code(
                            problem_description=problem_description,
                            current_code=python_code,
                            expected_behavior=expected_behavior,
                            current_incorrect_results=f"Returned type {type(result)}, but the result should be a string.",
                        ).fixed_code
                    )
                elif check_solution(sample_output, result)[  # Found a solutiond!
                    "matches"
                ]:
                    print(
                        f"CORRECT SOLN FOUND! CODE OPTION {i}/{len(python_solutions)} | DEBUGGING ITER: {try_iter}/{self.max_tries-1}."
                        )
                    return dspy.Prediction(solution=python_code)
                else:  # Otherwise, we should be able to check the solution
                    solution_results = check_solution(sample_output, result)
                    # with dspy.context(lm=gpt4):
                    python_code = extract_code(
                        self.fix_code(
                            problem_description=problem_description,
                            current_code=python_code,
                            expected_behavior=expected_behavior,
                            current_incorrect_results=format_mistakes(solution_results),
                        ).fixed_code
                    )
        return dspy.Prediction(solution=python_code)

### OPTIMIZATION ###

def optimize_with_mipro(program, prompt_model, task_model, metric, trainset):
    teleprompter = MIPROv2(
        prompt_model=prompt_model,
        task_model=task_model,
        metric=metric,
        num_candidates=5,
        init_temperature=0.5,
        verbose=False,
        log_dir="/lfs/0/kristaoo/dspy/examples/functional/logs",
    )

    optimized_program = teleprompter.compile(
        program.deepcopy(),
        trainset=trainset,
        eval_kwargs=dict(num_threads=16),
        max_bootstrapped_demos=0, # 0-shot optimization
        max_labeled_demos=0,
        num_batches=20,
        minibatch=False, # turning this off bc we have a small trainset already
        seed=9
    )

    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")

    optimized_program.save(f"mipro_optimized_{date_time}")

    return optimized_program

def optimize_with_bootstrap_fewshot(program, task_model, teacher_model, metric, trainset):
    rs_optimizer = BootstrapFewShotWithRandomSearch(
        metric=test_code(timeout=5),
        num_threads=8,
        num_candidate_programs=5,
        max_labeled_demos=0,
        max_bootstrapped_demos=2,
        max_errors =10000,
        teacher_settings=dict(lm=teacher_model)
    )

    optimized_program = rs_optimizer.compile(
        program,
        trainset=trainset,
    )

    now = datetime.now()
    date_time = now.strftime("%Y%m%d_%H%M%S")

    optimized_program.save(f"fewshot_optimized_{date_time}")


    return optimized_program

### DEFINING FUNCTION FOR TESTING CODE TO USE AS METRIC ###
### TODO: why this syntax?
def test_code(timeout=5):
    def metric(example, pred, trace=None):
        if pred.solution is None:
            return 0
        solution_code = pred.solution
        result_dict = run(
            code=solution_code, input=example.sample_input, timeout=timeout
        )
        if not result_dict["result"]:
            return 0
        return int(
            check_solution(example.sample_output, result_dict["result"])["matches"]
        )

    return metric


In [ ]:
### LOAD AND PREPARE DATA ###
ds = datasets.load_dataset("hackercupai/hackercup")

# Shuffle data
ds_full_list = list(ds["full"])
rng = random.Random(0)
rng.shuffle(ds_full_list)

# Format dataset to use in DSPy
# TODO: what does this syntax mean
sample_ds = [
    Example(
        problem_description=example["statement"],
        sample_input=example["sample_input"].strip().split("\n"),
        sample_output=example["sample_output"],
    ).with_inputs("problem_description", "sample_input", "sample_output")
    for example in ds["sample"]
    if example["sample_input"]
]

full_ds = [
    Example(
        problem_description=example["statement"],
        sample_input=example["sample_input"].strip().split("\n"),
        sample_output=example["sample_output"],
    ).with_inputs("problem_description", "sample_input", "sample_output")
    for example in ds_full_list
    if example["sample_input"]
]

trainset = sample_ds + full_ds[0:40] # use sample in train because it's easier
testset = full_ds[40:60]

# Configure our dspy settings (particularly LM we're using)
lm = dspy.OpenAI(
    model="gpt-4o-mini-2024-07-18", # Note: didn't find much a difference btwn mini & full gpt-4o
    max_tokens=4000,
    temperature=0.1,
)

dspy.settings.configure(lm=lm)
dspy.configure(experimental=True)

# Setup evaluation function
evaluate = Evaluate(
    devset=testset,
    num_threads=16, # Note: Set this to 1 for debugging purposes
    display_progress=True,
    display_table=5,
    metric=test_code(timeout=5)
)

In [ ]:
# Try out a simple program (7.5% on 40 ex)
simple_program = SimpleGenerateCode()
print("Evaluating Simple Program on test...")
evaluate(program=simple_program, devset=testset)

Evaluating Simple Program on test...


Average Metric: 3 / 20  (15.0): 100%|██████████| 20/20 [00:05<00:00,  3.85it/s]


,problem_description,sample_input,sample_output,solution,metric
0,"Alice and Bob are servers at *Nim Sum Dim Sum*, a bustling dumpling restaurant. For a staff meal, the manager has generously provided \(N\) plates...","['6', '3', '4 1 2', '2', '1 1', '2', '2 4', '3', '1 3 2', '6', '2 2 3 3 4 4', '8', '6 2...",Case #1: 6 Case #2: 0 Case #3: 2 Case #4: 0 Case #5: 0 Case #6: 19,"def solve(input: list) -> str: T = int(input[0]) results = [] index = 1 for case_number in range(1, T + 1): N = int(input[index]) A...",
1,"**Note: The only difference between this chapter and [chapter 2](https://www.facebook.com/codingcompetitions/hacker-cup/2022/round-1/problems/A2) is that here, all card values are guaranteed to be distinct and only up to...","['4', '5 1', '5 1 2 4 3', '2 4 3 5 1', '4 10', '3 1 4 2', '1 2 3 4', '4 0',...",Case #1: YES Case #2: NO Case #3: NO Case #4: YES,"def solve(input: list) -> str: T = int(input[0]) results = [] index = 1 for t in range(1, T + 1): N, K = map(int,...",
2,"In the rapidly growing towns of Silicon Valley, traffic congestion is becoming a major problem. The governor has contracted Travis, the rock star traffic engineer,...","['4', '2 2 999 999', '2 3 12 11', '4 3 6 6', '50 50 1 1']",Case #1: Possible 333 333 333 333 Case #2: Possible 5 3 1 3 4 3 Case #3: Possible 1 1 1 1 2 1...,"def solve(input: list) -> str: T = int(input[0]) results = [] for i in range(1, T + 1): N, M, A, B = map(int, input[i].split())...",
3,"Alfredo Spaghetti really likes soup, especially when it contains alphabet pasta. Every day he constructs a sentence from letters, places the letters into a bowl...","['5', 'WELCOME TO FACEBOOK HACKERCUP', 'CUP WITH LABEL HACKERCUP BELONGS TO HACKER', 'QUICK CUTE BROWN FOX JUMPS OVER THE LAZY DOG', 'MOVE FAST BE BOLD',...",Case #1: 1 Case #2: 2 Case #3: 1 Case #4: 0 Case #5: 1,"def solve(input: list) -> str: from collections import Counter target_word = ""HACKERCUP"" target_count = Counter(target_word) T = int(input[0]) results = [] for i in range(1,...",✔️ [1]
4,"Every week you take a socially-distanced trip to Overwaitea, your favourite grocery store, to stock up on food. Out front, there are \(N\) shopping carts,...","['5', '1', '2 3', '3', '4 5', '1 4', '5 1', '4', '3 2', '4 6', '5 5', '1 3', '6', '5 6', '3 5',...",Case #1: 11 3 2 Case #2: 190 20 0 16 30 Case #3: 560 0 5 0 24 36 Case #4: 1900 67 0...,"def solve(input: list) -> str: from collections import defaultdict T = int(input[0]) index = 1 results = [] for case_number in range(1, T + 1):...",


15.0

In [ ]:
# Try out more advanced pipeline | ~25%
multi_stage_program = GenerateCode(max_tries=3, num_ensembles=3)
print("Evaluating Multi-Stage Program on test...")
evaluate(program=multi_stage_program, devset=testset)

Evaluating Multi-Stage Program on test...


  0%|          | 0/20 [00:00<?, ?it/s]

Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 0
Case #2: 12
Case #3: 2500
Case #4: 499999888
Case #5: 83924361
Case #1: 1
Case #2: 0
Case #3: 0
Case #4: 1
Case #5: 5Case #1: NO
Case #2: NO
Case #3: NO
Case #4: YES

Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0
Case #5: 0
Case #6: 0
Case #1: 2
Case #2: 3
Case #3: -1
Case #1: 2
Case #2: 2
Case #3: 3
Case #4: 3
Case #5: 5
Case #1: 16
Case #2: 12
Case #3: 4
Case #4: 28
Case #1: 1
Case #2: 2
Case #3: 1
Case #4: 0
Case #5: 1
Case #1: 1 2
Case #2: 2 4 6 8
Case #3: 3 6 9 12 15
Case #4: 4 8 12 16 20 24 28
Case #5: 5 10 15 20 25 30 35 40 45 50 55Case #1: NYYY
Case #2: YNYYY
Case #3: YYYY
Case #4: NNYYYYNNY
Case #5: YYYYYYYYYN
Case #6: NNNNNNYNNNNNNNNNNNNNNNNNNNNYNN

Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 5
Case #2: 14
Case #3: 20
CORRECT SOLN FOUND! CODE OPTION 0/3 | DEBUGGING ITER: 0/2.
Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 5
Case #2: 14
Case #3: 20
Case #1: 28

Average Metric: 1 / 1  (100.0):   5%|▌         | 1/20 [00:08<02:34,  8.11s/it]

Case #1: -1
Case #2: -1
Case #3: 4
Case #4: -1
Case #5: -1
Case #1: 28
Case #2: 32
Case #3: 32
Case #4: 809
Case #5: 4600
Case #1: 2
Case #2: 3
Case #3: 4
Case #4: 4
Case #5: -1
Case #1: 28
Case #2: 32
Case #3: 32
Case #4: 809
Case #5: 4600
Case #1: 2
Case #2: 3
Case #3: 4
Case #4: 4
Case #5: -1
Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 10
Case #2: 10
Case #3: 10
Case #4: 1
Case #5: 1000
Case #1: 2
Case #2: 5
Case #3: -1
Case #4: -1
Case #5: -1


...

Average Metric: 5 / 19  (26.3):  95%|█████████▌| 19/20 [01:02<00:03,  3.40s/it]

Case #1: Possible
998 1
998 1
Case #2: Possible
10 1 1
9 1 1
Case #3: Possible
2 1 1
1 1 1
1 1 1
2 1 1
Case #4: Impossible


Average Metric: 5 / 20  (25.0): 100%|██████████| 20/20 [01:03<00:00,  3.15s/it]


,problem_description,sample_input,sample_output,solution,metric
0,"Alice and Bob are servers at *Nim Sum Dim Sum*, a bustling dumpling restaurant. For a staff meal, the manager has generously provided \(N\) plates...","['6', '3', '4 1 2', '2', '1 1', '2', '2 4', '3', '1 3 2', '6', '2 2 3 3 4 4', '8', '6 2...",Case #1: 6 Case #2: 0 Case #3: 2 Case #4: 0 Case #5: 0 Case #6: 19,"def solve(input): T = int(input[0]) results = [] for i in range(1, T + 1): N = int(input[2 * i - 1]) A = list(map(int,...",
1,"**Note: The only difference between this chapter and [chapter 2](https://www.facebook.com/codingcompetitions/hacker-cup/2022/round-1/problems/A2) is that here, all card values are guaranteed to be distinct and only up to...","['4', '5 1', '5 1 2 4 3', '2 4 3 5 1', '4 10', '3 1 4 2', '1 2 3 4', '4 0',...",Case #1: YES Case #2: NO Case #3: NO Case #4: YES,"def solve(input): T = int(input[0]) results = [] index = 1 for t in range(1, T + 1): N, K = map(int, input[index].split()) A =...",
2,"In the rapidly growing towns of Silicon Valley, traffic congestion is becoming a major problem. The governor has contracted Travis, the rock star traffic engineer,...","['4', '2 2 999 999', '2 3 12 11', '4 3 6 6', '50 50 1 1']",Case #1: Possible 333 333 333 333 Case #2: Possible 5 3 1 3 4 3 Case #3: Possible 1 1 1 1 2 1...,"def solve(input): from sys import stdout input = input[1:] # Skip the first line which is T results = [] for case_number, line in enumerate(input,...",
3,"Alfredo Spaghetti really likes soup, especially when it contains alphabet pasta. Every day he constructs a sentence from letters, places the letters into a bowl...","['5', 'WELCOME TO FACEBOOK HACKERCUP', 'CUP WITH LABEL HACKERCUP BELONGS TO HACKER', 'QUICK CUTE BROWN FOX JUMPS OVER THE LAZY DOG', 'MOVE FAST BE BOLD',...",Case #1: 1 Case #2: 2 Case #3: 1 Case #4: 0 Case #5: 1,"def solve(input): from collections import Counter # The target word we want to form target_word = ""HACKERCUP"" # Count the frequency of letters in the...",✔️ [1]
4,"Every week you take a socially-distanced trip to Overwaitea, your favourite grocery store, to stock up on food. Out front, there are \(N\) shopping carts,...","['5', '1', '2 3', '3', '4 5', '1 4', '5 1', '4', '3 2', '4 6', '5 5', '1 3', '6', '5 6', '3 5',...",Case #1: 11 3 2 Case #2: 190 20 0 16 30 Case #3: 560 0 5 0 24 36 Case #4: 1900 67 0...,"def solve(input): from collections import defaultdict T = int(input[0]) index = 1 results = [] for case_number in range(1, T + 1): N = int(input[index])...",


25.0

In [ ]:
# Try out more advanced pipeline | ~30%
multi_stage_program = GenerateCode(max_tries=5, num_ensembles=5)
print(f"Evaluating Multi-Stage Program on test...")
evaluate(program=multi_stage_program, devset=testset)

Evaluating Multi-Stage Program on test...


  0%|          | 0/20 [00:00<?, ?it/s]

Case #1: 1
Case #2: 0
Case #3: 1
Case #4: 0
Case #5: 0
Case #6: 0
Case #1: NO
Case #2: NO
Case #3: NO
Case #4: NO
Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 2
Case #5: 2
Case #6: 4
Case #7: 8
Case #1: 2
Case #2: 3
Case #3: -1
Case #1: 0
Case #2: 3
Case #3: 30
Case #4: 360
Case #5: 3420Case #1: 2
Case #2: 3
Case #3: 3
Case #4: 7
Case #5: 31

Case #1: 2
Case #2: 1
Case #3: 1
Case #4: 2
Case #5: 5
Case #1: 10
Case #2: 10
Case #3: 32
Case #4: 1
Case #5: 4600Case #1: Possible
333 333
333 333
Case #2: Possible
3 3 3
3 3 3
Case #3: Possible
1 1 1
1 1 1
1 1 1
1 1 1
Case #4: Impossible

Case #1: 1
Case #2: 2
Case #3: 1
Case #4: 0
Case #5: 1Case #1: 0
Case #2: 1
Case #3: 0
Case #4: 8
Case #5: 6
Case #6: 3
Case #7: 104

Case #1: 0
Case #2: 1
Case #3: 0
Case #4: 8
Case #5: 6
Case #6: 3
Case #7: 104
Case #1: 0
Case #2: 0
Case #3: 5
Case #4: 0
Case #1: 4
Case #2: 5
Case #3: 4
Case #4: 1
Case #5: 6Case #1: NYYY
Case #2: YNYNY
Case #3: NYYY
Case #4: NYNNYYNNY
Case #5: NYNNNYNYYN
Case #6: NNNNNNNNNNNYNN

Average Metric: 1 / 1  (100.0):   5%|▌         | 1/20 [00:07<02:18,  7.30s/it]

Case #1: -1
Case #2: -1
Case #3: 2
Case #4: -1
Case #5: 8
Case #1: 5
Case #2: 5
Case #3: 7
Case #4: 1
Case #5: 10
Case #1: 2
Case #2: 5
Case #3: 6
Case #4: 7
Case #5: -1
Case #1: 2
Case #2: 5
Case #3: 6
Case #4: 7
Case #5: -1
Case #1: 1
Case #2: 2
Case #3: 1
Case #4: 0
Case #5: 1
CORRECT SOLN FOUND! CODE OPTION 0/5 | DEBUGGING ITER: 1/4.
CORRECT SOLN FOUND! CODE OPTION 0/5 | DEBUGGING ITER: 1/4.
Case #1: 2
Case #2: 5
Case #3: 6
Case #4: 7
Case #5: -1
Case #1: Possible
333 333
333 333
Case #2: Possible
3 3 3
3 3 3
Case #3: Possible
1 1 1
1 1 1
1 1 1
1 1 1
Case #4: Impossible
Case #1: 1
Case #2: 2
Case #3: 1
Case #4: 0
Case #5: 1
Case #1: 4
Case #2: 4
Case #3: 4
Case #4: 3
Case #5: 9
Case #1: 0
Case #2: 1
Case #3: 0
Case #4: 8
Case #5: 6
Case #6: 3
Case #7: 104


...

Average Metric: 5.0 / 19  (26.3):  95%|█████████▌| 19/20 [02:52<00:10, 10.44s/it]

Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0
Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0
Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0
Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0
Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0
Case #1: 3 2
Case #2: 3 2 1 0
Case #3: 3 2 3 1 0
Case #4: 3 2 10 6 3 1 0
Case #5: 3 2 36 28 21 15 10 6 3 1 0


Average Metric: 5.0 / 20  (25.0): 100%|██████████| 20/20 [02:54<00:00,  8.72s/it]


,problem_description,sample_input,sample_output,solution,metric
0,"Alice and Bob are servers at *Nim Sum Dim Sum*, a bustling dumpling restaurant. For a staff meal, the manager has generously provided \(N\) plates...","['6', '3', '4 1 2', '2', '1 1', '2', '2 4', '3', '1 3 2', '6', '2 2 3 3 4 4', '8', '6 2...",Case #1: 6 Case #2: 0 Case #3: 2 Case #4: 0 Case #5: 0 Case #6: 19,"def nim_sum(a): result = 0 for num in a: result ^= num return result def calculate_winning_moves(N, A): total_winning_moves = 0 for i in range(N): for...",
1,"**Note: The only difference between this chapter and [chapter 2](https://www.facebook.com/codingcompetitions/hacker-cup/2022/round-1/problems/A2) is that here, all card values are guaranteed to be distinct and only up to...","['4', '5 1', '5 1 2 4 3', '2 4 3 5 1', '4 10', '3 1 4 2', '1 2 3 4', '4 0',...",Case #1: YES Case #2: NO Case #3: NO Case #4: YES,"def solve(input): T = int(input[0]) results = [] index = 1 for t in range(1, T + 1): N, K = map(int, input[index].split()) A =...",
2,"In the rapidly growing towns of Silicon Valley, traffic congestion is becoming a major problem. The governor has contracted Travis, the rock star traffic engineer,...","['4', '2 2 999 999', '2 3 12 11', '4 3 6 6', '50 50 1 1']",Case #1: Possible 333 333 333 333 Case #2: Possible 5 3 1 3 4 3 Case #3: Possible 1 1 1 1 2 1...,"def solve(input): T = int(input[0]) results = [] for case_number in range(1, T + 1): N, M, A, B = map(int, input[case_number].split()) grid = [[1]...",
3,"Alfredo Spaghetti really likes soup, especially when it contains alphabet pasta. Every day he constructs a sentence from letters, places the letters into a bowl...","['5', 'WELCOME TO FACEBOOK HACKERCUP', 'CUP WITH LABEL HACKERCUP BELONGS TO HACKER', 'QUICK CUTE BROWN FOX JUMPS OVER THE LAZY DOG', 'MOVE FAST BE BOLD',...",Case #1: 1 Case #2: 2 Case #3: 1 Case #4: 0 Case #5: 1,"def solve(input): from collections import Counter # The target word target_word = ""HACKERCUP"" target_count = Counter(target_word) # Prepare the output list output = [] T...",✔️ [1.0]
4,"Every week you take a socially-distanced trip to Overwaitea, your favourite grocery store, to stock up on food. Out front, there are \(N\) shopping carts,...","['5', '1', '2 3', '3', '4 5', '1 4', '5 1', '4', '3 2', '4 6', '5 5', '1 3', '6', '5 6', '3 5',...",Case #1: 11 3 2 Case #2: 190 20 0 16 30 Case #3: 560 0 5 0 24 36 Case #4: 1900 67 0...,"def solve(input): from collections import defaultdict T = int(input[0]) results = [] index = 1 for case_number in range(1, T + 1): N = int(input[index])...",


25.0

In [ ]:
# OPTIONAL: Optimize program w/ MIPROv2 (0-shot)
multi_stage_program = GenerateCode()
mipro_optimized_multi_stage_program = optimize_with_mipro(multi_stage_program, lm, lm, test_code(timeout=5), trainset)
print(f"Evaluating MIPRO optimized Multi-Stage Program on test...")
evaluate(program=mipro_optimized_multi_stage_program, devset=testset)

TypeError: MIPROv2.compile() got an unexpected keyword argument 'eval_kwargs'

In [ ]:
# OPTIONAL: Optimize program w/ Bootstrap Few-Shot
multi_stage_program = GenerateCode()
bootstrap_optimized_multi_stage_program = optimize_with_bootstrap_fewshot(multi_stage_program, lm, lm, test_code(timeout=5), trainset)
print(f"Evaluating Bootstrap Few-Shot optimized Multi-Stage Program on test...")
evaluate(program=bootstrap_optimized_multi_stage_program, devset=testset)

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 5 candidate sets.
Case #1: 4
Case #2: 2
Case #3: 1

  0%|          | 0/50 [00:00<?, ?it/s]


Case #1: -1
Case #2: -1
Case #3: 1
Case #4: -1
Case #5: -1
Case #6: -1
Case #7: -1
Case #1: 0
Case #2: 2
Case #1: YES
Case #2: NO
Case #3: YES
Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0
Case #1: YES
Case #2: NO
Case #3: YES
Case #4: YES
Case #5: YES
Case #6: YES
Case #7: NO
Case #1: -1
Case #2: -1
Case #3: 1
Case #4: -1
Case #5: -1
Case #6: -1
Case #7: -1
CORRECT SOLN FOUND! CODE OPTION 0/3 | DEBUGGING ITER: 0/1.
Case #1: NO
Case #2: YES
Case #3: NO
Case #1: 0
Case #2: 0
Case #3: 0
Case #1: 6
Case #2: 2
Case #1: YES
Case #2: NO
Case #3: YES
Case #4: YES
Case #5: YES
Case #6: YES
Case #7: NO
Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0
Case #1: YES
Case #2: YES
Case #3: NO
Case #4: NO
Case #5: YES


Average Metric: 1 / 1  (100.0):   2%|▏         | 1/50 [00:04<03:29,  4.28s/it]

CORRECT SOLN FOUND! CODE OPTION 0/3 | DEBUGGING ITER: 1/1.CORRECT SOLN FOUND! CODE OPTION 0/3 | DEBUGGING ITER: 1/1.

Case #1: YES
Case #2: YES
Case #3: NO
Case #4: NO
Case #5: YES
Case #1: NO
Case #2: YES
Case #3: NO
Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0
Case #1: 3
Case #2: 0
Case #1: 0
Case #2: 0
Case #3: 0

...

Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0
Case #1: 0
Case #2: 0
Case #3: 0
Case #4: 0


Average Metric: 10 / 50  (20.0): 100%|██████████| 50/50 [02:35<00:00,  3.12s/it]


Scores so far: [20.0, 20.0]
Best score so far: 20.0


  0%|          | 0/50 [00:00<?, ?it/s]ERROR:dspy.teleprompt.bootstrap:2024-09-25T06:50:03.345730Z [error    ] Failed to run or to evaluate example Example({'problem_description': 'Hacker Cup contest strategy often involves a metagame, where choosing which problems to work on might just be an important decision. On a Quest to become more Pro, you encounter an oracle promising to teach you the contest meta if you play her own Meta-game.\n\nThe oracle presents a peg board with \\(2N\\) moving dots. The initial \\(y\\)-positions of the dots are given as two arrays \\(A_{1..N}\\) and \\(B_{1..N}\\). Each second, simultaneously, \\(A_1\\) will move to the end of \\(B\\), while \\(B_1\\) will move to the end of \\(A\\) (with all elements shifting left accordingly).\n\nYou can connect the dots to form a *Meta-like logo* if all of the following are true:\n* For the first half of both arrays, each dot in \\(A\\) is below the corresponding dot in \\(B\\).\n* For the last half of both arrays, each

RuntimeError: asyncio.run() cannot be called from a running event loop